In [1]:
import xarray as xr
import numpy as np
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

In [ ]:
case = "fanv3_improving"
path = "/glade/scratch/jinmuluo/archive/nitrogen_cycle/hist/" 
start_date = date(2010, 1, 1)
end_date = date(2010, 12, 1)
delta = end_date - start_date

GAS=['NH3_TOTAL', 'NH3_GRZ', 'NH3_BARNS', 'NH3_STORES', 'NH3_FERT', 'area', 'landfrac']

CLM = []
for i in range(delta.days//30 + 1):
    month = start_date + relativedelta(months=i)
    month = month.strftime('%Y-%m')
    CLM.append(path + case + ".clm2." + "h0" + "." + month +".nc4")